# Projet Intelligence Artificielle

## Besoin client 3 : Système d'alerte pour les tempêtes

### Préparation des données

In [21]:
import pandas as pd

data_prof = pd.read_csv('./Données/Data_Arbre.csv')
data_clean = pd.read_csv("./Données/data_clean.csv", encoding='utf-8', sep=";", decimal=",")

In [23]:
data_prof.head()
# data_clean.head()

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non


In [ ]:
data = data_prof[["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","fk_stadedev","age_estim", "fk_prec_estim"]]

# autre méthode

colonnes = ["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","fk_stadedev","age_estim", "fk_prec_estim"]
data_new = data_prof[colonnes]

In [25]:
#corrélation
correlation = data_prof.corr(method='pearson', numeric_only= True).style.background_gradient(cmap='Pastel1')
correlation

,longitude,latitude,haut_tot,haut_tronc,tronc_diam,age_estim,fk_prec_estim,clc_nbr_diag
longitude,1.000000,-0.020745,0.152302,0.087468,0.077484,-0.028906,0.004257,-0.032664
latitude,-0.020745,1.000000,0.125222,0.054681,0.008317,-0.015870,-0.072914,-0.041084
haut_tot,0.152302,0.125222,1.000000,0.496415,0.686894,0.601767,0.496149,0.396635
haut_tronc,0.087468,0.054681,0.496415,1.000000,0.361884,0.474419,0.368215,0.336130
tronc_diam,0.077484,0.008317,0.686894,0.361884,1.000000,0.779173,0.607214,0.361216
age_estim,-0.028906,-0.015870,0.601767,0.474419,0.779173,1.000000,0.758507,0.373180
fk_prec_estim,0.004257,-0.072914,0.496149,0.368215,0.607214,0.758507,1.000000,0.366723
clc_nbr_diag,-0.032664,-0.041084,0.396635,0.336130,0.361216,0.373180,0.366723,1.000000
